# ЗАДАНИЕ ПО ВЫЧМАТАМ. ВАРИАНТ 7.5

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# Суточные колебания концентрации озона в атмосфере Рассмотрим простейшую математическую модель колебаний концентрации озона в ат- мосфере. Она описывается следующей неавтономной системой ОДУ: 

Параметры системы, начальные условия

In [2]:
omega = np.pi/43200  #c-1
k1, k2 = 1.63e-16, 4.66e-16
c3, c4 = 22.62, 7.601

In [3]:
t = np.arange(0, 4*43200, 1)  # step of 1 second
def light_dep(w, c, m):  # m for fit in velocities
    res = np.array(0)  # for the case, when it's darkness all the time
    for _ in t:
        if np.sin(w*_) <= 0.0:
            continue
        elif round(np.exp(-c/np.sin(w*_))*m):  # not to use ky less than 2 score of k2y1y3
            res = np.append(res, round(np.exp(-c/np.sin(w*_))*10*m, 2))  # exponent >= -10m
    res = np.delete(res, 0)
    return res
    

In [4]:
k3, k4 = light_dep(omega, c4, 1e15), light_dep(omega, c4, 1e11)  # e-16 and e-12 accordingly

In [5]:
y1_0, y2_0, y3_0 = 1e6, 3.7e16, 1e12  # см-3 

In [1]:
class Onestepmethod (object):
    def __init__(self, f, y0, t0, te, N, tol):
        self.f = f
        self.y0 = y0.astype(float)
        self.t0 = t0
        self.interval = [t0, te]
        self.grid = linspace(t0, te, N+2)
        self.h = (te-t0)/(N+1)
        self.N = N
        self.tol = tol
        self.m = len(y0)
        self.s = len(self.b)
        self.solution = None

    def step(self):
        ti, yi = self.grid[0], self.y0
        tim1 = ti
        yield np.hstack((array([ti]), yi))
        for ti in self.grid[1:]:
            yi = yi + self.h * self.phi(tim1, yi)
            tim1 = ti
            yield np.hstack((array([ti]), yi))

    def solve(self):
        self.solution = list(self.step())

    # To be implemented in a derived class
    def phi(self, tim1, yi):
        return 1

In [2]:
class RungeImplicit(Onestepmethod):
    def phi(self, t0, y0):
        M = 10
        stageDer = array(self.s*[self.f(t0,y0)])
        J = jacobian(self.f, t0, y0)
        stageVal = self.phi_solve(t0, y0, stageDer, J, M)
        return array([dot(self.b, stageVal.reshape(self.s,self.m)[:, j]) for j in range(self.m)])

    def phi_solve(self, t0, y0, initVal, J, M):
        JJ = eye(self.s*self.m)-self.h*np.kron(self.A, J)
        luFactor = linalg.lu_factor(JJ)
        for i in range(M):
            initVal, norm_d = self.phi_newtonstep(t0, y0, initVal, luFactor)
            if norm_d < self.tol:
                # print('Newton converged in {} steps'.format(i))
                break
            elif i == M-1:
                raise ValueError('The Newton iteration did not converge.')
        return initVal

    def phi_newtonstep(self, t0, y0, initVal, luFactor):
        d = linalg.lu_solve(luFactor, - self.F(initVal.flatten(), t0, y0))
        return initVal.flatten() + d, norm(d)

    def F(self, stageDer, t0, y0):
        stageDer_new = empty((self.s, self.m))
        for i in range(self.s):
            stageVal = y0 + array([self.h * dot(self.A[i, :],
                                  stageDer.reshape(self.s, self.m)[:, j]) for j in range(self.m)])
            stageDer_new[i, :] = self.f(t0 + self.c[i] * self.h, stageVal)
        return stageDer - stageDer_new.reshape(-1)

In [11]:
class Ordinate:
    def __init__(self, y1=y1_0, y2=y2_0, y3=y3_0):
        self.y1 = y1
        self.y2 = y2
        self.y3 = y3
    def __str__(self):
        return set(y1, y2, y3)

    
y = Ordinate()

In [13]:
def kineq(y, t):  
    y1 = y[0]
    y2 = y[1]
    y3 = y[2]
    dydt_1 = -k1*y1*y2 - k2*y1*y3 + 2*k3*y2 + k4*y3
    # dydt_2 = 0
    dydt_3 = k1*y1*y2 - k2*y1*y3 - k4*y3
    return set(dydt_1, dydt_3)

In [15]:
t = np.linspace(0, 4*43200)
y_1 = odeint(kineq, y1_0, t, axis=2)
y_3 = odeint(kineq, y3_0, t, axis=2)

TypeError: odeint() got an unexpected keyword argument 'axis'